<a href="https://colab.research.google.com/github/SMTorg/smt-design-space-ext/blob/master/tutorial/SMT_DesignSpace_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="jumbotron text-left"><b>
    
This tutorial describes how to use de DesignSpace within the SMT toolbox. 
<div>
    
    September 2025 - `SMT version 2.9.4`
  
     Paul Saves (IRIT), Jasper Bussemaker (DLR), and Nathalie BARTOLI (ONERA/DTIS/M2CI)

<div class="alert alert-info fade in" id="d110">
<p>Some updates</p>
<ol> -  Manipulation of mixed DOE (continuous, integer,  categorical and hierarchical variables) </ol>
</div>

<p class="alert alert-success" style="padding:1em">
To use SMT models, please follow this link : https://github.com/SMTorg/SMT/blob/master/README.md. The documentation is available here: http://smt.readthedocs.io/en/latest/
</p>

The reference paper is available 
here https://www.sciencedirect.com/science/article/pii/S096599782300162X



For mixed integer with continuous relaxation, the reference paper is available here https://www.sciencedirect.com/science/article/pii/S0925231219315619

In [1]:
# to have the latest version
!pip install smt==2.9.4
!pip install configspace==0.6.1
!pip install git+https://github.com/SMTorg/smt-design-space-ext.git

  Cloning https://github.com/jbussemaker/adsg-core.git (to revision dev) to c:\users\psaves\appdata\local\temp\pip-req-build-8mnp6ins
  Resolved https://github.com/jbussemaker/adsg-core.git to commit 0f426bd871fe7bf2f466f5f4dab26cd9304a6b9b
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for adsg-core: filename=adsg_core-1.2.0-py3-none-any.whl size=191402 sha256=d8d5beeaf3c2682f21169a51643e6a757700e73e3e612c665b6f188a55bbf9be
  Stored in directory: C:\Users\psaves\AppData\Local\Temp\pip-ephem-wheel-cache-34tteuqa\wheels\5b\f4\a0\6bf1efdbf6bd3c1987ea76e8e7a2c3b2668ae8963bcb02193c
Successfully built adsg-core
  Attempting uninstall: adsg-core
    Found existing installation: adsg-core 1.1.1
    

  Running command git clone --filter=blob:none --quiet https://github.com/jbussemaker/adsg-core.git 'C:\Users\psaves\AppData\Local\Temp\pip-req-build-8mnp6ins'
  Running command git checkout -b dev --track origin/dev
  branch 'dev' set up to track 'origin/dev'.
  Switched to a new branch 'dev'
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
smt-design-space-ext 0.3.1 requires adsg-core==1.1.1, but you have adsg-core 1.2.0 which is incompatible.


  Using cached adsg_core-1.1.1-py3-none-any.whl.metadata (6.3 kB)
Using cached adsg_core-1.1.1-py3-none-any.whl (189 kB)
  Attempting uninstall: adsg-core
    Found existing installation: adsg-core 1.2.0
    Uninstalling adsg-core-1.2.0:
      Successfully uninstalled adsg-core-1.2.0


<div class="alert alert-warning" >
If you use hierarchical variables and the size of your doe greater than 30 points, you may leverage the `numba` JIT compiler to speed up the computation
To do so:
    
 - install numba library
    
     `pip install numba`
    
    
 - and define the environment variable `USE_NUMBA_JIT = 1` (unset or 0 if you do not want to use numba) 
    
     - Linux: export USE_NUMBA_JIT = 1
    
     - Windows: set USE_NUMBA_JIT = 1

</div>

In [2]:
%matplotlib inline

from smt_design_space_ext import (
    AdsgDesignSpaceImpl,
    ConfigSpaceDesignSpaceImpl,
    DesignSpace,
    FloatVariable,
    IntegerVariable,
    OrdinalVariable,
    CategoricalVariable,
)

import plotly.io as pio

# to ignore warning messages
import warnings

warnings.filterwarnings("ignore")

pio.renderers.default = "notebook"

# Manipulate DOE with mixed, categorical & hierarchical variables

4 variables 
 - 1 categorical variable with 2 labels ['A', 'B'] # x0 categorical: A or B; order is not relevant
 - 1 ordinal variable with 3 levels ['C', 'D', 'E']),  # x1 ordinal: C, D or E; order is relevant
 - 1 integer variable [0,2]: 3 possibilities: 0, 1, 2
 - 1 continuous variable $\in [0, 1]$
 
 
 **Posssibility to have hierarchical variable: x1 exists only if x0 = 'A'**

In [3]:
# Instantiate the design space with all its design variables:

ds = DesignSpace(
    [
        CategoricalVariable(
            ["A", "B"]
        ),  # x0 categorical: A or B; order is not relevant
        OrdinalVariable(["C", "D", "E"]),  # x1 ordinal: C, D or E; order is relevant
        IntegerVariable(0, 2),  # x2 integer between 0 and 2 (inclusive): 0, 1, 2
        FloatVariable(0, 1),  # c3 continuous between 0 and 1
    ]
)

print("Number of design variables", len(ds.design_variables))
# You can define decreed variables (conditional activation):
ds.declare_decreed_var(
    decreed_var=1, meta_var=0, meta_value="A"
)  # Activate x1 if x0 == A

Number of design variables 4


In [4]:
## To give some examples
# It is also possible to randomly sample design vectors conforming to the constraints:
n = 5
x_sampled, is_acting_sampled = ds.sample_valid_x(5)

print("Data encoded: \n", x_sampled)
print("Data in initial space: \n", ds.decode_values(x_sampled))

Data encoded: 
 [[0.         2.         1.         0.61498752]
 [0.         0.         0.         0.27522602]
 [1.         0.         1.         0.9050206 ]
 [1.         0.         1.         0.07916433]
 [0.         0.         2.         0.48073098]]
Data in initial space: 
 [['A', 'E', 1.0, 0.6149875242755137], ['A', 'C', 0.0, 0.2752260217303657], ['B', 'C', 1.0, 0.9050205954557852], ['B', 'C', 1.0, 0.07916432721737987], ['A', 'C', 2.0, 0.4807309794918219]]


In [5]:
# After defining everything correctly, you can then use the design space object
# to correct design vectors and get information about which design variables are acting:
x_corr, is_acting = ds.correct_get_acting(x_sampled)
print("Which variables are active \n", is_acting)

Which variables are active 
 [[ True  True  True  True]
 [ True  True  True  True]
 [ True False  True  True]
 [ True False  True  True]
 [ True  True  True  True]]


In [6]:
# If needed, it is possible to get the legacy design space definition format:
xlimits = ds.get_x_limits()
cont_bounds = ds.get_num_bounds()
unfolded_cont_bounds = ds.get_unfolded_num_bounds()
print("Limits of each variable \n", xlimits)
print("Continuous bounds with the encoding done (4 variables now) \n", cont_bounds)
print(
    "Continuous bounds with the unfolded encoding done (5 variables now)\n",
    unfolded_cont_bounds,
)

Limits of each variable 
 [['A', 'B'], ['0', '1', '2'], (0, 2), (0, 1)]
Continuous bounds with the encoding done (4 variables now) 
 [[0 1]
 [0 2]
 [0 2]
 [0 1]]
Continuous bounds with the unfolded encoding done (5 variables now)
 [[0. 1.]
 [0. 1.]
 [0. 2.]
 [0. 2.]
 [0. 1.]]


# Manipulate DOE with continuous variables

In [7]:
# You can also instantiate a purely-continuous design space from bounds directly:
continuous_design_space = DesignSpace([(0, 1), (0, 2), (0.5, 5.5)])
print(
    "Number of design variables =",
    continuous_design_space.n_dv,
    " or ",
    len(continuous_design_space.design_variables),
)

Number of design variables = 3  or  3


In [8]:
x_sampled_cont, is_acting_sampled_cont = continuous_design_space.sample_valid_x(5)

In [9]:
print("Data encoded: \n", x_sampled_cont)
print("Is_acting: \n", is_acting_sampled_cont)

Data encoded: 
 [[0.62965671 0.0536643  2.61133895]
 [0.41194587 1.65384962 1.75101379]
 [0.94626397 1.36758061 3.85140911]
 [0.0224535  0.51706134 1.32031797]
 [0.23434939 0.85952266 5.24856387]]
Is_acting: 
 [[ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]]


# Moving towards Architecture Design Space Graph (ADSG)

## First, let use ADSG

In [10]:
from adsg_core import BasicADSG, NamedNode, DesignVariableNode

# Create the ADSG
adsg = BasicADSG()

ndv = 13


# Create nodes
n = [NamedNode(f"N{i}") for i in range(ndv)]
n = [
    NamedNode("MLP"),
    NamedNode("Learning_rate"),
    NamedNode("Activation_function"),
    NamedNode("Optimizer"),
    NamedNode("Decay"),
    NamedNode("Power_update"),
    NamedNode("Average_start"),
    NamedNode("Running_Average_1"),
    NamedNode("Running_Average_2"),
    NamedNode("Numerical_Stability"),
    NamedNode("Nb_layers"),
    NamedNode("Layer_1"),
    NamedNode("Layer_2"),
    NamedNode("Layer_3"),  # NamedNode("Dropout"),
    NamedNode("ASGD"),
    NamedNode("Adam"),
    NamedNode("20...40"),
    NamedNode("40"),
    NamedNode("45"),
    NamedNode("20...40"),
    NamedNode("40"),
    NamedNode("45"),
    NamedNode("20...40"),
    NamedNode("40"),
    NamedNode("45"),
]
adsg.add_node(n[1])
adsg.add_node(n[2])
# adsg.add_node(n[3])

# Add some edges
adsg.add_edges(
    [
        # (n[0], n[1]),
        # (n[0], n[2]),
        # (n[0], n[3]),
        (n[3], n[10]),
        (n[14], n[4]),
        (n[14], n[5]),
        (n[14], n[6]),
        (n[15], n[7]),
        (n[15], n[8]),
        (n[15], n[9]),
    ]
)

choiceo = adsg.add_selection_choice("Optimizer_Choice", n[3], [n[14], n[15]])


choicenl = adsg.add_selection_choice("#layers", n[10], [n[11], n[12], n[13]])
# adsg.add_edges([ (n[12],n[11]), (n[13],n[12]) ])
a = []
for i in range(3):
    a.append(NamedNode(str(25 + 5 * i)))
b = a.copy()
b.append(n[17])
b.append(n[18])
choicel1 = adsg.add_selection_choice("#neurons_1", n[11], b)
adsg.add_edges([(n[12], choicel1), (n[13], choicel1)])


a = []
for i in range(3):
    a.append(NamedNode(str(25 + 5 * i)))
b = a.copy()
b.append(n[20])
b.append(n[21])
choicel1 = adsg.add_selection_choice("#neurons_2", n[12], b)
adsg.add_edges([(n[13], choicel1)])

a = []
for i in range(3):
    a.append(NamedNode(str(25 + 5 * i)))
b = a.copy()
b.append(n[23])
b.append(n[24])
choicel1 = adsg.add_selection_choice("#neurons_3", n[13], b)

# adsg.add_edges([ (n[18],n[17]), (n[17],n[16]) ])
# adsg.add_edges([ (n[21],n[20]), (n[20],n[19]) ])
# adsg.add_edges([ (n[24],n[23]), (n[23],n[22]) ])

adsg.add_incompatibility_constraint([n[15], n[13]])
adsg.add_incompatibility_constraint([n[14], n[17]])
adsg.add_incompatibility_constraint([n[14], n[18]])
adsg.add_incompatibility_constraint([n[14], n[20]])
adsg.add_incompatibility_constraint([n[14], n[21]])
adsg.add_incompatibility_constraint([n[14], n[23]])
adsg.add_incompatibility_constraint([n[14], n[24]])
start_nodes = set()
start_nodes.add(n[3])
start_nodes.add(n[2])
start_nodes.add(n[1])
# start_nodes.add(n[0])


adsg.add_edges(
    [
        (n[1], DesignVariableNode("x0", bounds=(0, 1))),
        (n[4], DesignVariableNode("x1", bounds=(0, 1))),
        (n[5], DesignVariableNode("x2", bounds=(0, 1))),
        (n[6], DesignVariableNode("x3", bounds=(0, 1))),
        (n[7], DesignVariableNode("x4", bounds=(0, 1))),
        (n[8], DesignVariableNode("x5", bounds=(0, 1))),
        (n[9], DesignVariableNode("x6", bounds=(0, 1))),
        # (n[11], DesignVariableNode('x7', options=("0", "1"))),
    ]
)

choiceo = adsg.add_selection_choice(
    "Activation_Choice",
    n[2],
    [NamedNode("ReLU"), NamedNode("Sigmoid"), NamedNode("Tanh")],
)

adsg = adsg.set_start_nodes(start_nodes)
adsg.render()

## ADSG also comes with processing tools

In [11]:
from adsg_core import GraphProcessor

gp = GraphProcessor(adsg)

print("Design variables:", gp.des_vars[0:5])
print(str(gp.des_vars[5:])[1:])
print("Objectives:", gp.objectives)
print("Constraints:", gp.constraints)

# Display some details about the encoders used for
# formulating the optimization problem
gp.get_statistics()

Design variables: [DV: #layers [3 opts], DV: Activation_Choice [3 opts], DV: Optimizer_Choice [2 opts], DV: #neurons_1 [5 opts], DV: #neurons_2 [5 opts]]
DV: #neurons_3 [5 opts], DV: x0 [0.00..1.00], DV: x1 [0.00..1.00], DV: x2 [0.00..1.00], DV: x3 [0.00..1.00], DV: x4 [0.00..1.00], DV: x5 [0.00..1.00], DV: x6 [0.00..1.00]]
Objectives: []
Constraints: []


,n_valid,n_declared,n_discrete,n_dim_cont,n_dim_cont_mean,n_exist,imp_ratio,imp_ratio_comb,imp_ratio_cont,inf_idx,dist_corr,encoder
type,,,,,,,,,,,,
option-decisions,207,1350,6,0,0.0,1,6.521739,6.521739,1.00,0.531986,0.0,complete
additional-dvs,207,0,0,7,4.0,207,1.750000,1.000000,1.75,1.000000,0.0,
total-design-space,207,2250,6,7,4.0,1,19.021739,10.869565,1.75,0.493305,0.0,complete
total-design-problem,207,2250,6,7,4.0,1,19.021739,10.869565,1.75,0.493305,0.0,complete


## Now, let use the SMT interface of ADSG to sample points

In [12]:
design_space2 = AdsgDesignSpaceImpl(adsg=adsg)
design_space2._sample_valid_x(1, return_render=True)[2][0].render()

## One can also use SMT ConfigSpace, with the same API as before

In [13]:
# Define the mixed hierarchical design space
design_space3 = ConfigSpaceDesignSpaceImpl(
    [
        FloatVariable(0, 1),  # Learning rate
        CategoricalVariable(
            ["ReLU", "Sigmoid", "Tanh"]
        ),  # 3 possible choices for the activation function
        CategoricalVariable(["ASGD", "Adam"]),  # 2 possible choices for the optimizer
        FloatVariable(0, 1),  # ASGD Decay
        FloatVariable(0, 1),  # ASGD Power update
        FloatVariable(0, 1),  # ASGD Average start
        FloatVariable(0, 1),  # Adam Running Average 1
        FloatVariable(0, 1),  # Adam Running Average 2
        FloatVariable(0, 1),  # Adam Numerical Stability
        IntegerVariable(1, 3),  # for the number of hidden layers  (l=x9)
        OrdinalVariable(
            ["25", "30", "35", "40", "45"]
        ),  # number of hidden neurons layer 1
        OrdinalVariable(
            ["25", "30", "35", "40", "45"]
        ),  # number of hidden neurons layer 2
        OrdinalVariable(
            ["25", "30", "35", "40", "45"]
        ),  # number of hidden neurons layer 3
    ]
)

# ASGD vs Adam optimizer options activated or deactivated
design_space3.declare_decreed_var(decreed_var=3, meta_var=2, meta_value=["ASGD"])
design_space3.declare_decreed_var(decreed_var=4, meta_var=2, meta_value=["ASGD"])
design_space3.declare_decreed_var(decreed_var=5, meta_var=2, meta_value=["ASGD"])
design_space3.declare_decreed_var(decreed_var=6, meta_var=2, meta_value=["Adam"])
design_space3.declare_decreed_var(decreed_var=7, meta_var=2, meta_value=["Adam"])
design_space3.declare_decreed_var(decreed_var=8, meta_var=2, meta_value=["Adam"])

# Number of hidden layers: Activate x11 when x9 in [2, 3] and x12 when x9 == 3
design_space3.add_value_constraint(
    var1=9, value1=3, var2=2, value2=["Adam"]
)  # Forbid 3 hidden layers with Adam
design_space3.declare_decreed_var(decreed_var=10, meta_var=9, meta_value=[1, 2, 3])
design_space3.declare_decreed_var(decreed_var=11, meta_var=9, meta_value=[2, 3])
design_space3.declare_decreed_var(decreed_var=12, meta_var=9, meta_value=3)
design_space3.add_value_constraint(
    var1=10, value1=["40", "45"], var2=2, value2=["ASGD"]
)  # Forbid more than 35 neurons with ASGD
design_space3.add_value_constraint(
    var1=11, value1=["40", "45"], var2=2, value2=["ASGD"]
)  # Forbid more than 35 neurons with ASGD
design_space3.add_value_constraint(
    var1=12, value1=["40", "45"], var2=2, value2=["ASGD"]
)  # Forbid more than 35 neurons with ASGD

In [14]:
x_sampled, is_acting_sampled = design_space3.sample_valid_x(2)
print("Sampled data: \n", x_sampled)

Sampled data: 
 [[0.53373847 1.         0.         0.62715152 0.07777511 0.81469579
  0.5        0.5        0.5        1.         2.         0.
  0.        ]
 [0.84677387 0.         1.         0.5        0.5        0.5
  0.14880918 0.27022458 0.09812246 1.         2.         0.
  0.        ]]


## Or with ADSG

In [15]:
# Define the mixed hierarchical design space
design_space4 = AdsgDesignSpaceImpl(
    design_variables=[
        FloatVariable(0, 1),  # Learning rate
        CategoricalVariable(
            ["ReLU", "Sigmoid", "Tanh"]
        ),  # 3 possible choices for the activation function
        CategoricalVariable(["ASGD", "Adam"]),  # 2 possible choices for the optimizer
        FloatVariable(0, 1),  # ASGD Decay
        FloatVariable(0, 1),  # ASGD Power update
        FloatVariable(0, 1),  # ASGD Average start
        FloatVariable(0, 1),  # Adam Running Average 1
        FloatVariable(0, 1),  # Adam Running Average 2
        FloatVariable(0, 1),  # Adam Numerical Stability
        OrdinalVariable(["1", "2", "3"]),  # for the number of hidden layers  (l=x9)
        OrdinalVariable(
            ["25", "30", "35", "40", "45"]
        ),  # number of hidden neurons layer 1
        OrdinalVariable(
            ["25", "30", "35", "40", "45"]
        ),  # number of hidden neurons layer 2
        OrdinalVariable(
            ["25", "30", "35", "40", "45"]
        ),  # number of hidden neurons layer 3
    ]
)

# ASGD vs Adam optimizer options activated or deactivated
design_space4.declare_decreed_var(decreed_var=3, meta_var=2, meta_value=["ASGD"])
design_space4.declare_decreed_var(decreed_var=4, meta_var=2, meta_value=["ASGD"])
design_space4.declare_decreed_var(decreed_var=5, meta_var=2, meta_value=["ASGD"])
design_space4.declare_decreed_var(decreed_var=6, meta_var=2, meta_value=["Adam"])
design_space4.declare_decreed_var(decreed_var=7, meta_var=2, meta_value=["Adam"])
design_space4.declare_decreed_var(decreed_var=8, meta_var=2, meta_value=["Adam"])

# Number of hidden layers: Activate x11 when x9 in [2, 3] and x12 when x9 == 3
design_space4.add_value_constraint(
    var1=9, value1="3", var2=2, value2=["Adam"]
)  # Forbid 3  hidden layers with Adam
design_space4.declare_decreed_var(
    decreed_var=10, meta_var=9, meta_value=["1", "2", "3"]
)
design_space4.declare_decreed_var(decreed_var=11, meta_var=9, meta_value=["2", "3"])
design_space4.declare_decreed_var(decreed_var=12, meta_var=9, meta_value="3")
design_space4.add_value_constraint(
    var1=10, value1=["40", "45"], var2=2, value2=["ASGD"]
)  # Forbid more than 35 neurons with ASGD
design_space4.add_value_constraint(
    var1=11, value1=["40", "45"], var2=2, value2=["ASGD"]
)  # Forbid more than 35 neurons with ASGD
design_space4.add_value_constraint(
    var1=12, value1=["40", "45"], var2=2, value2=["ASGD"]
)  # Forbid more than 35 neurons with ASGD

In [21]:
design_space4._sample_valid_x(1, return_render=True)[2][0].render()
design_space4._sample_valid_x(1, return_render=True)[2][0].render()
design_space4._sample_valid_x(1, return_render=True)[2][0].render()

# Use smt to build a surrogate model

In this section, we use a simpler test case of only 8 dimensions, the simpler input space consists of:  

- **Ordinal:** `x0 ∈ {1, 2, 3}` (meta variable), `x3 ∈ {8, 16, 32, 64, 128, 256}`  
- **Continuous:** `x1, x2 ∈ [-5, 2]`  
- **Categorical:** `x4 ∈ {ReLU, SELU, ISRLU}`  
- **Integer:** `x5, x6, x7 ∈ [0, 5]`  (decreed variables included or not depending on $x0$ as  `x6` is active if `x0 ≥ 2` and `x7` is active if `x0 = 3`  


In [17]:
import numpy as np
from smt.surrogate_models import KRG
from smt.surrogate_models import MixIntKernelType
from smt.design_space import (
    DesignSpace,
    FloatVariable,
    IntegerVariable,
    OrdinalVariable,
    CategoricalVariable,
)
from smt.problems import HierarchicalNeuralNetwork as nn

problem = nn()

# One can access the predifed design space as:
ds = problem.design_space
# or alternatively, redefine it like that:

design_space = AdsgDesignSpaceImpl(
    design_variables=[
        OrdinalVariable(values=["1", "2", "3"]),  # x0
        FloatVariable(-5, 2),
        FloatVariable(-5, 2),
        OrdinalVariable(values=[8, 16, 32, 64, 128, 256]),  # x3
        CategoricalVariable(values=["ReLU", "SELU", "ISRLU"]),  # x4
        IntegerVariable(0, 5),  # x5
        IntegerVariable(0, 5),  # x6
        IntegerVariable(0, 5),  # x7
    ]
)

# x6 is active when x0 >= 2
design_space.declare_decreed_var(decreed_var=6, meta_var=0, meta_value=["2", "3"])
# x7 is active when x0 >= 3
design_space.declare_decreed_var(decreed_var=7, meta_var=0, meta_value=["3"])
design_space.adsg.render()
n_doe = 160
xdoe = problem.sample(n_doe)
y = problem(xdoe)

In [18]:
# Surrogate
sm = KRG(
    design_space=ds,
    print_global=False,
    categorical_kernel=MixIntKernelType.GOWER,
)

sm.set_training_values(xdoe, y)
sm.train()

In [19]:
print(
    "interpolation error: ",
    100 * np.sum(np.abs((sm.predict_values(xdoe[:])[:, 0] - y[:]) / (160 * y[:]))),
    "%",
)

interpolation error:  8.859716046334922e-10 %


In [20]:
n_doe = 20
xdoe = problem.sample(n_doe)
y = problem(xdoe)
print(
    "prediction error: ",
    100 * np.sum(np.abs((sm.predict_values(xdoe[:])[:, 0] - y[:]) / (20 * y[:]))),
    "%",
)

prediction error:  10.957415105823607 %
